In [ ]:
#things to toggle on the interface

#parametrize:
# the room dimensions and wall curvature
# gravity
# air type and density
# shape of the ball
# material properties of the ball
# initial conditions


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pydantic import BaseModel, Field

class Ball(BaseModel):
    """a 3d ball with material and state"""
    radius: float
    mass: float
    color: str = "blue"
    #material properties
    elasticity: float
    density: float
    dilation: float = 0.0

    #initial conditions
    start_position: list[float]
    start_velocity: list[float]
    current_position: np.ndarray = Field(default_factory=lambda: np.zeros(3, dtype=float))
    current_velocity: np.ndarray = Field(default_factory=lambda: np.zeros(3, dtype=float))
    acceleration: np.ndarray = Field(default_factory=lambda: np.zeros(3, dtype=float))
    force: np.ndarray = Field(default_factory=lambda: np.zeros(3, dtype=float))

    class Config:
        arbitrary_types_allowed = True

    def initialize_state(self) -> None:
        self.current_position = np.array(self.start_position, dtype=float)
        self.current_velocity = np.array(self.start_velocity, dtype=float)
        self.acceleration = np.zeros(3, dtype=float)
        self.force = np.zeros(3, dtype=float)


medium_types = ["air", "helium", "nitrogen"]
class BounceEnvironment(BaseModel):
    """environment parameters for the room"""

    #room features
    room_dimensions: list[float]
    gravity: float
    gravity_direction: list[float] = [0.0, -1.0, 0.0]
    linear_drag: float = 0.0
    quadratic_drag: float = 0.0
    medium_type: str = "air"
    restitution: float = 0.9
    wall_curvature: float = 0.0
    air_density: float = 1.225

    class Config:
        arbitrary_types_allowed = True


class Simulation(BaseModel):
    """run a physics simulation for multiple balls"""
    balls: list[Ball]
    environment: BounceEnvironment
    time_step: float
    total_time_steps: int
    positions: list[np.ndarray] = Field(default_factory=list)
    velocities: list[np.ndarray] = Field(default_factory=list)

    class Config:
        arbitrary_types_allowed = True

    def _gravity_vector(self) -> np.ndarray:
        g_dir = np.array(self.environment.gravity_direction, dtype=float)
        g_norm = np.linalg.norm(g_dir)
        if g_norm == 0.0:
            return np.zeros(3, dtype=float)
        return (g_dir / g_norm) * self.environment.gravity

    def calculate_forces(self) -> None:
        g_vec = self._gravity_vector()
        for ball in self.balls:
            ball.force = ball.mass * g_vec

            v = ball.current_velocity
            speed = np.linalg.norm(v)
            if self.environment.linear_drag != 0.0:
                ball.force += -self.environment.linear_drag * v
            if self.environment.quadratic_drag != 0.0 and speed > 0.0:
                ball.force += -self.environment.quadratic_drag * speed * v

    def update_acceleration(self) -> None:
        for ball in self.balls:
            ball.acceleration = ball.force / ball.mass

    def update_velocity(self) -> None:
        for ball in self.balls:
            ball.current_velocity = (
                ball.current_velocity + ball.acceleration * self.time_step
            )

    def update_position(self) -> None:
        for ball in self.balls:
            ball.current_position = (
                ball.current_position + ball.current_velocity * self.time_step
            )

    def update_deformation(self) -> None:
        for ball in self.balls:
            speed = np.linalg.norm(ball.current_velocity)
            ball.dilation = ball.dilation + speed * self.time_step

    def bounce_off_walls(self, ball: Ball) -> None:
        dims = np.array(self.environment.room_dimensions, dtype=float)
        pos = ball.current_position
        vel = ball.current_velocity
        r = ball.radius
        restitution = self.environment.restitution

        for axis in range(3):
            min_bound = r
            max_bound = dims[axis] - r
            if pos[axis] < min_bound:
                pos[axis] = min_bound
                vel[axis] = -vel[axis] * restitution
            elif pos[axis] > max_bound:
                pos[axis] = max_bound
                vel[axis] = -vel[axis] * restitution

        ball.current_position = pos
        ball.current_velocity = vel

    def handle_ball_collisions(self) -> None:
        restitution = self.environment.restitution
        count = len(self.balls)

        for i in range(count):
            for j in range(i + 1, count):
                b1 = self.balls[i]
                b2 = self.balls[j]
                delta = b2.current_position - b1.current_position
                dist = np.linalg.norm(delta)
                min_dist = b1.radius + b2.radius

                if dist >= min_dist:
                    continue

                if dist == 0.0:
                    normal = np.array([1.0, 0.0, 0.0], dtype=float)
                else:
                    normal = delta / dist

                overlap = min_dist - dist
                b1.current_position -= normal * (overlap / 2.0)
                b2.current_position += normal * (overlap / 2.0)

                rel_vel = b2.current_velocity - b1.current_velocity
                vel_along_normal = np.dot(rel_vel, normal)
                if vel_along_normal > 0.0:
                    continue

                inv_m1 = 0.0 if b1.mass == 0.0 else 1.0 / b1.mass
                inv_m2 = 0.0 if b2.mass == 0.0 else 1.0 / b2.mass
                impulse_mag = (-(1.0 + restitution) * vel_along_normal) / (inv_m1 + inv_m2)
                impulse = impulse_mag * normal

                b1.current_velocity -= impulse * inv_m1
                b2.current_velocity += impulse * inv_m2

    def _snapshot(self) -> tuple[np.ndarray, np.ndarray]:
        positions = np.stack([b.current_position for b in self.balls], axis=0)
        velocities = np.stack([b.current_velocity for b in self.balls], axis=0)
        return positions, velocities

    def step(self) -> None:
        self.calculate_forces()
        self.update_acceleration()
        self.update_velocity()
        self.update_position()
        self.update_deformation()
        for ball in self.balls:
            self.bounce_off_walls(ball)
        self.handle_ball_collisions()
        positions, velocities = self._snapshot()
        self.positions.append(positions)
        self.velocities.append(velocities)

    def simulate(self) -> None:
        for ball in self.balls:
            ball.initialize_state()
        positions, velocities = self._snapshot()
        self.positions = [positions]
        self.velocities = [velocities]
        for _ in range(self.total_time_steps):
            self.step()


/var/folders/37/mbm5qm1d03lg63qfy97k4tmw0000gn/T/ipykernel_14053/1904150842.py:5: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Ball(BaseModel):
/var/folders/37/mbm5qm1d03lg63qfy97k4tmw0000gn/T/ipykernel_14053/1904150842.py:28: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class BounceEnvironment(BaseModel):
/var/folders/37/mbm5qm1d03lg63qfy97k4tmw0000gn/T/ipykernel_14053/1904150842.py:44: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  

In [6]:
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display


def _random_non_overlapping_positions(count, radius, room_dims, max_tries=5000):
    rng = np.random.default_rng()
    positions = []
    mins = np.array([radius, radius, radius], dtype=float)
    maxs = np.array(room_dims, dtype=float) - radius

    for _ in range(count):
        placed = False
        for _ in range(max_tries):
            candidate = rng.uniform(mins, maxs)
            if all(np.linalg.norm(candidate - p) >= 2.0 * radius for p in positions):
                positions.append(candidate)
                placed = True
                break
        if not placed:
            raise ValueError("could not place all balls without overlap")
    return positions


def _build_balls(count, radius, mass, room_dims, speed):
    positions = _random_non_overlapping_positions(count, radius, room_dims)
    rng = np.random.default_rng()
    balls = []
    for i in range(count):
        direction = rng.normal(size=3)
        direction = direction / np.linalg.norm(direction)
        velocity = direction * speed
        balls.append(
            Ball(
                radius=radius,
                mass=mass,
                start_position=positions[i].tolist(),
                start_velocity=velocity.tolist(),
                color="blue",
            )
        )
    return balls


def _plot_positions(positions, room_dims, frame_stride=1):
    dims = np.array(room_dims, dtype=float)
    frames = []

    for t in range(0, len(positions), frame_stride):
        pos = positions[t]
        frames.append(
            go.Frame(
                data=[
                    go.Scatter3d(
                        x=pos[:, 0],
                        y=pos[:, 1],
                        z=pos[:, 2],
                        mode="markers",
                        marker=dict(size=5, color="blue"),
                    )
                ],
                name=str(t),
            )
        )

    initial = positions[0]
    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=initial[:, 0],
                y=initial[:, 1],
                z=initial[:, 2],
                mode="markers",
                marker=dict(size=5, color="blue"),
            )
        ],
        frames=frames,
    )

    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[0, dims[0]]),
            yaxis=dict(range=[0, dims[1]]),
            zaxis=dict(range=[0, dims[2]]),
            aspectmode="data",
        ),
        margin=dict(l=0, r=0, b=0, t=30),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(
                        label="play",
                        method="animate",
                        args=[None, {"frame": {"duration": 30, "redraw": True}}],
                    ),
                    dict(
                        label="pause",
                        method="animate",
                        args=[[None], {"frame": {"duration": 0}, "mode": "immediate"}],
                    ),
                ],
            )
        ],
    )

    return fig


ball_count = widgets.IntSlider(value=5, min=1, max=30, step=1, description="balls")
ball_radius = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description="radius")
ball_mass = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="mass")
ball_speed = widgets.FloatSlider(value=1.0, min=0.0, max=5.0, step=0.1, description="speed")

room_x = widgets.FloatSlider(value=10.0, min=3.0, max=30.0, step=0.5, description="room x")
room_y = widgets.FloatSlider(value=10.0, min=3.0, max=30.0, step=0.5, description="room y")
room_z = widgets.FloatSlider(value=10.0, min=3.0, max=30.0, step=0.5, description="room z")

gravity = widgets.FloatSlider(value=9.81, min=0.0, max=20.0, step=0.1, description="gravity")
restitution = widgets.FloatSlider(value=0.9, min=0.0, max=1.0, step=0.05, description="restitution")
linear_drag = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.05, description="linear drag")
quadratic_drag = widgets.FloatSlider(value=0.0, min=0.0, max=2.0, step=0.05, description="quadratic drag")

time_step = widgets.FloatSlider(value=0.02, min=0.005, max=0.1, step=0.005, description="dt")
steps = widgets.IntSlider(value=300, min=50, max=2000, step=50, description="steps")
frame_stride = widgets.IntSlider(value=2, min=1, max=10, step=1, description="stride")

run_button = widgets.Button(description="run simulation")
output = widgets.Output()


def _run_simulation(_):
    output.clear_output(wait=True)
    with output:
        dims = [room_x.value, room_y.value, room_z.value]
        env = BounceEnvironment(
            room_dimensions=dims,
            gravity=gravity.value,
            gravity_direction=[0.0, -1.0, 0.0],
            linear_drag=linear_drag.value,
            quadratic_drag=quadratic_drag.value,
            restitution=restitution.value,
        )
        balls = _build_balls(
            ball_count.value,
            ball_radius.value,
            ball_mass.value,
            dims,
            ball_speed.value,
        )
        sim = Simulation(
            balls=balls,
            environment=env,
            time_step=time_step.value,
            total_time_steps=steps.value,
        )
        sim.simulate()
        fig = _plot_positions(sim.positions, dims, frame_stride=frame_stride.value)
        display(fig)


run_button.on_click(_run_simulation)

controls = widgets.VBox(
    [
        ball_count,
        ball_radius,
        ball_mass,
        ball_speed,
        room_x,
        room_y,
        room_z,
        gravity,
        restitution,
        linear_drag,
        quadratic_drag,
        time_step,
        steps,
        frame_stride,
        run_button,
    ]
)

ui = widgets.HBox([controls, output])
display(ui)



In [ ]:
#things to add:
#energy/entropoy of the balls by color codes
#monte carlo simulation of the grid space that is most entered by the balls